In [245]:
using Pkg
Pkg.activate(".")

  Activating project at `c:\Users\fefif\Desktop\PhD\Progetti\MatrixProductBP.jl`


In [246]:
using Revise
using MatrixProductBP, MatrixProductBP.Models
using Graphs, IndexedGraphs, Statistics, Random, LinearAlgebra, TensorTrains, SparseArrays
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using TensorTrains: summary_compact
using SparseArrays
using JLD2
include("bp_fourier.jl");

In [247]:
seed = 1
rng = MersenneTwister(seed)

T = 4
N = 3
A = [0 1 1; 
     1 0 0;
     1 0 0;]
# N = 4
# A = [0 1 1 1;
#      1 0 0 1;
#      1 0 0 1;
#      1 1 1 0]
# !!! Here the calculations are wrong for edge 1->2 !!!
# N = 5
# A = [0 1 1 1 1; 
#      1 0 0 0 0; 
#      1 0 0 0 0; 
#      1 0 0 0 0; 
#      1 0 0 0 0]
# N = 6
# A = [0 1 1 1 0 0; 
#      1 0 0 0 1 0; 
#      1 0 0 0 0 1; 
#      1 0 0 0 0 0; 
#      0 1 0 0 0 0; 
#      0 0 1 0 0 0]
# N = 7
# A = [0 1 1 1 1 0 1;
#      1 0 0 0 1 0 0; 
#      1 0 0 0 0 1 0; 
#      1 0 0 0 0 0 0; 
#      1 1 0 0 0 0 0; 
#      0 0 1 0 0 0 0; 
#      1 0 0 0 0 0 0]

g = IndexedBiDiGraph(A)

# N = 10
# c = 3
# gg = erdos_renyi(N, c/N; seed)
# g = IndexedBiDiGraph(gg)
# A = zeros(Int,N,N)
# for i in CartesianIndices(A)
#      if !iszero(g.A[i])
#          A[i] = 1
#      end
# end
# rand!(rng, A.nzval, (-1,1))  # fill directed incidence matrix with rand(-1,1)
β = 1.0
J = 1
h = 0.0
m⁰ = 1.0
K = 100

svd_trunc=TruncBond(4)

A

3×3 Matrix{Int64}:
 0  1  1
 1  0  0
 1  0  0

In [248]:
w = [fill(IntegerGlauberFactor([J*A[j,i] for j in inneighbors(g,i)], h, β), T+1) for i in vertices(g)]
ϕᵢ = [t == 0 ? [(1-m⁰)/2, (1+m⁰)/2] : ones(2) for t in 0:T]
bp = mpbp(g, w, fill(2,N), T; ϕ=fill(ϕᵢ,N))

for μ in bp.μ
    for a in μ
        a.=rand.(rng)
        # a.=1.0
    end
end
iterate!(bp, maxiter=1; svd_trunc)
# msg = deepcopy(bp.μ)
# jldsave("messages.jld2"; msg)

bp_fourier = deepcopy(bp)
;

In [249]:
iterate!(bp,maxiter=1; svd_trunc)

(1, CB_BP{ProgressMeter.ProgressUnknown, MatrixProductBP.var"#153#156"}(ProgressMeter.ProgressUnknown(false, false, 0, ProgressMeter.ProgressCore(:green, "Running MPBP: iter", 0.1, true, 0, VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(Base.Libc.WindowsRawSocket(0x0000000000000358) open, 0 bytes waiting)), VSCodeServer.io_send_callback), false, 1, 1, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 0, 416299675879)), 0, 1, false, false, 1.734623053179e9, 1.734623053179e9, 1.734623053179e9)), [[[2.0, 1.2933802728735118, 1.4504346458632182, 1.668915890476069, 1.2339699967042288], [2.0, 1.9128847560458306, 1.7910954054951012, 1.7191197163386156, 1.594783356867103], [2.0, 1.830371096615877, 1.5323947111854859, 1.4233596020706587, 1.6160851691378215]], [[2.0, 1.9820137900379085, 1.8670988855829598, 1

In [250]:
iterate_fourier!(bp_fourier,K, maxiter=1, σ=1/100; svd_trunc)
;

In [251]:
import MatrixProductBP: marginalize
function marginalize(A::MPEM2{F}, var) where F
    if var==1
        return MPEM1{F}([@tullio b[m, n, xi] := a[m, n, xi, xj] for a in A]; z = A.z)
    elseif var==2
        return MPEM1{F}([@tullio b[m, n, xj] := a[m, n, xi, xj] for a in A]; z = A.z)
    end
end;

In [252]:
er = []
for e in edges(bp.g)
    any(any(isnan, a) for a in bp_fourier.μ[idx(e)]) && error("NaN in messages")
    μ_marg = marginalize(bp.μ[idx(e)],1)
    margs = marginals(μ_marg)
    μ_fourier_marg = marginalize(bp_fourier.μ[idx(e)],1)
    margs_fourier = marginals(μ_fourier_marg)
    for t in 1:T+1
        marg = margs[t]
        marg_fourier = margs_fourier[t]
        ind = argmax(abs.(marg))
        push!(er, abs(marg_fourier[ind]-marg[ind])/abs(marg[ind]))
    end
end

display(maximum(er))
display(argmax(er))

1.4216240118738086

3

In [266]:
node = 2
time = 5

ed = inedges(bp.g, node) |> first
@show ed
μ = bp.μ[idx(ed)]
μ_marg = marginalize(μ,1)
marg = marginals(μ_marg)[time]

ed_fourier = inedges(bp_fourier.g, node) |> first
@show ed_fourier
μ_fourier = bp_fourier.μ[idx(ed_fourier)]
μ_fourier_marg = marginalize(μ_fourier,1)
marg_fourier = marginals(μ_fourier_marg)[time]

display(marg)
display(marg_fourier);

2-element Vector{Float64}:
 0.43261920410187465
 0.5673807958981254

2-element Vector{Float64}:
 0.8397973083602855
 0.16020269163971448

ed = Indexed Edge 1 => 2 with index 1
ed_fourier = Indexed Edge 1 => 2 with index 1


In [254]:
for e in edges(bp.g)
    display(e)
end

Indexed Edge 1 => 2 with index 1

Indexed Edge 1 => 3 with index 2

Indexed Edge 2 => 1 with index 3

Indexed Edge 3 => 1 with index 4